# Predicting sentiment from product reviews


The goal of this first notebook is to explore logistic regression and feature engineering with existing Turi Create functions.

In this notebook you will use product review data from Amazon.com to predict whether the sentiments about a product (from its reviews) are positive or negative.

* Use SFrames to do some feature engineering
* Train a logistic regression model to predict the sentiment of product reviews.
* Inspect the weights (coefficients) of a trained logistic regression model.
* Make a prediction (both class and probability) of sentiment for a new product review.
* Given the logistic regression weights, predictors and ground truth labels, write a function to compute the **accuracy** of the model.
* Inspect the coefficients of the logistic regression model and interpret their meanings.
* Compare multiple logistic regression models.

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt

## Load Data

In [2]:
products = pd.read_csv('amazon_baby.csv')
products

,name,review,rating
0,Planetwise Flannel Wipes,"These flannel wipes are OK, but in my opinion ...",3
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5
2,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,5
3,Stop Pacifier Sucking without tears with Thumb...,This is a product well worth the purchase. I ...,5
4,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5
...,...,...,...
183526,Baby Teething Necklace for Mom Pretty Donut Sh...,Such a great idea! very handy to have and look...,5
183527,Baby Teething Necklace for Mom Pretty Donut Sh...,This product rocks! It is a great blend of fu...,5
183528,Abstract 2 PK Baby / Toddler Training Cup (Pink),This item looks great and cool for my kids.......,5
183529,"Baby Food Freezer Tray - Bacteria Resistant, B...",I am extremely happy with this product. I have...,5


## Build the word count vector for each review

Now, we will perform 2 simple data transformations:

1. Remove punctuation using [Python's built-in](https://docs.python.org/2/library/string.html) string functionality.
2. Transform the reviews into word-counts.

**Aside**. In this notebook, we remove all punctuations for the sake of simplicity. A smarter approach to punctuations would preserve phrases such as "I'd", "would've", "hadn't" and so forth. See [this page](https://www.cis.upenn.edu/~treebank/tokenization.html) for an example of smart handling of punctuations.

In [3]:
import string
import re

In [4]:
# Function that cleans text by removing '\x0c' and '\n' characters
# as well as all non-alpha characters and finally converts everything
# to lower case

def clean_text(text):
    #translator = text.maketrans('', '', string.punctuation)
    #clean_text = text.translate(translator)
    # remove punctuation
    text = re.sub(r'[^\w\s]', ' ', text)
    # remove all non-alpha characters and converts to lower case
    text = re.sub('[^a-zA-Z]+', ' ', text).lower()
    
    return text

test_str = products['review'][0]
print(clean_text(test_str))

these flannel wipes are ok but in my opinion not worth keeping i also ordered someimse vimse cloth wipes ocean blue countwhich are larger had a nicer softer texture and just seemed higher quality i use cloth wipes for hands and faces and have been usingthirsties pack fab wipes boyfor about months now and need to replace them because they are starting to get rough and have had stink issues for a while that stripping no longer handles 


**IMPORTANT**. Make sure to fill n/a values in the **review** column with empty strings (if applicable). The n/a values indicate empty reviews. For instance, Pandas's the fillna() method lets you replace all N/A's in the **review** columns as follows:

In [5]:
products = products.fillna({'review':''})  # fill in N/A's in the review column
products['review_clean'] = products['review'].apply(clean_text)
products

,name,review,rating,review_clean
0,Planetwise Flannel Wipes,"These flannel wipes are OK, but in my opinion ...",3,these flannel wipes are ok but in my opinion n...
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5,it came early and was not disappointed i love ...
2,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,5,very soft and comfortable and warmer than it l...
3,Stop Pacifier Sucking without tears with Thumb...,This is a product well worth the purchase. I ...,5,this is a product well worth the purchase i ha...
4,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5,all of my kids have cried non stop when i trie...
...,...,...,...,...
183526,Baby Teething Necklace for Mom Pretty Donut Sh...,Such a great idea! very handy to have and look...,5,such a great idea very handy to have and look ...
183527,Baby Teething Necklace for Mom Pretty Donut Sh...,This product rocks! It is a great blend of fu...,5,this product rocks it is a great blend of func...
183528,Abstract 2 PK Baby / Toddler Training Cup (Pink),This item looks great and cool for my kids.......,5,this item looks great and cool for my kids i k...
183529,"Baby Food Freezer Tray - Bacteria Resistant, B...",I am extremely happy with this product. I have...,5,i am extremely happy with this product i have ...


Now, let us explore what the sample example above looks like after these 2 transformations. Here, each entry in the **word_count** column is a dictionary where the key is the word and the value is a count of the number of times the word occurs.

In [6]:
# add an empty column to dataframe using assign
products['word_count'] = ''

for i in range(len(products)):
    d = {}
    
    if pd.isna(products['review'][i]):
        products["word_count"][i] = d
    else:
        word_list = [word.strip(",.") for word in products['review'][i].lower().split()]
        for word in word_list:
            if word not in d:
                d[word] = 1
            d[word] += 1
        products["word_count"][i] = d

products

C:\Users\yc1033514\Anaconda3\envs\py37\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


,name,review,rating,review_clean,word_count
0,Planetwise Flannel Wipes,"These flannel wipes are OK, but in my opinion ...",3,these flannel wipes are ok but in my opinion n...,"{'these': 2, 'flannel': 2, 'wipes': 4, 'are': ..."
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5,it came early and was not disappointed i love ...,"{'it': 4, 'came': 2, 'early': 2, 'and': 4, 'wa..."
2,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,5,very soft and comfortable and warmer than it l...,"{'very': 2, 'soft': 2, 'and': 3, 'comfortable'..."
3,Stop Pacifier Sucking without tears with Thumb...,This is a product well worth the purchase. I ...,5,this is a product well worth the purchase i ha...,"{'this': 5, 'is': 5, 'a': 3, 'product': 3, 'we..."
4,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5,all of my kids have cried non stop when i trie...,"{'all': 3, 'of': 2, 'my': 2, 'kids': 3, 'have'..."
...,...,...,...,...,...
183526,Baby Teething Necklace for Mom Pretty Donut Sh...,Such a great idea! very handy to have and look...,5,such a great idea very handy to have and look ...,"{'such': 2, 'a': 2, 'great': 3, 'idea!': 2, 'v..."
183527,Baby Teething Necklace for Mom Pretty Donut Sh...,This product rocks! It is a great blend of fu...,5,this product rocks it is a great blend of func...,"{'this': 2, 'product': 3, 'rocks!': 2, 'it': 3..."
183528,Abstract 2 PK Baby / Toddler Training Cup (Pink),This item looks great and cool for my kids.......,5,this item looks great and cool for my kids i k...,"{'this': 3, 'item': 2, 'looks': 2, 'great': 3,..."
183529,"Baby Food Freezer Tray - Bacteria Resistant, B...",I am extremely happy with this product. I have...,5,i am extremely happy with this product i have ...,"{'i': 10, 'am': 3, 'extremely': 2, 'happy': 2,..."


## Extract sentiments

We will **ignore** all reviews with *rating = 3*, since they tend to have a neutral sentiment.

In [7]:
products = products[products['rating'] != 3]
len(products)

166752

Now, we will assign reviews with a rating of 4 or higher to be *positive* reviews, while the ones with rating of 2 or lower are *negative*. For the sentiment column, we use +1 for the positive class label and -1 for the negative class label.

In [8]:
products['sentiment'] = products['rating'].apply(lambda rating : +1 if rating > 3 else -1)
products

C:\Users\yc1033514\Anaconda3\envs\py37\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,name,review,rating,review_clean,word_count,sentiment
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5,it came early and was not disappointed i love ...,"{'it': 4, 'came': 2, 'early': 2, 'and': 4, 'wa...",1
2,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,5,very soft and comfortable and warmer than it l...,"{'very': 2, 'soft': 2, 'and': 3, 'comfortable'...",1
3,Stop Pacifier Sucking without tears with Thumb...,This is a product well worth the purchase. I ...,5,this is a product well worth the purchase i ha...,"{'this': 5, 'is': 5, 'a': 3, 'product': 3, 'we...",1
4,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5,all of my kids have cried non stop when i trie...,"{'all': 3, 'of': 2, 'my': 2, 'kids': 3, 'have'...",1
5,Stop Pacifier Sucking without tears with Thumb...,"When the Binky Fairy came to our house, we did...",5,when the binky fairy came to our house we didn...,"{'when': 3, 'the': 7, 'binky': 4, 'fairy': 4, ...",1
...,...,...,...,...,...,...
183526,Baby Teething Necklace for Mom Pretty Donut Sh...,Such a great idea! very handy to have and look...,5,such a great idea very handy to have and look ...,"{'such': 2, 'a': 2, 'great': 3, 'idea!': 2, 'v...",1
183527,Baby Teething Necklace for Mom Pretty Donut Sh...,This product rocks! It is a great blend of fu...,5,this product rocks it is a great blend of func...,"{'this': 2, 'product': 3, 'rocks!': 2, 'it': 3...",1
183528,Abstract 2 PK Baby / Toddler Training Cup (Pink),This item looks great and cool for my kids.......,5,this item looks great and cool for my kids i k...,"{'this': 3, 'item': 2, 'looks': 2, 'great': 3,...",1
183529,"Baby Food Freezer Tray - Bacteria Resistant, B...",I am extremely happy with this product. I have...,5,i am extremely happy with this product i have ...,"{'i': 10, 'am': 3, 'extremely': 2, 'happy': 2,...",1


## Split data into training and test sets

In [9]:
X = products['review_clean']
y = products['sentiment']

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

# Train a sentiment classifier with logistic regression

We will now use logistic regression to create a sentiment classifier on the training data. This model will use the column **word_count** as a feature and the column **sentiment** as the target. We will use `validation_set=None` to obtain same results as everyone else.

We will now compute the word count for each word that appears in the reviews. A vector consisting of word counts is often referred to as **bag-of-word features**. Since most words occur in only a few reviews, word count vectors are sparse. For this reason, scikit-learn and many other tools use sparse matrices to store a collection of word count vectors. Refer to appropriate manuals to produce sparse word count vectors. General steps for extracting word count vectors are as follows:

* Learn a vocabulary (set of all words) from the training data. Only the words that show up in the training data will be considered for feature extraction.
* Compute the occurrences of the words in each review and collect them into a row vector.
* Build a sparse matrix where each row is the word count vector for the corresponding review. Call this matrix **train_matrix**.
* Using the same mapping between words and columns, convert the test data into a sparse matrix **test_matrix**.

In [11]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(token_pattern=r'\b\w+\b')
# Use this token pattern to keep single-letter words
# First, learn vocabulary from the training data and assign columns to words
# Then convert the training data into a sparse matrix
train_matrix = vectorizer.fit_transform(X_train)
# Second, convert the test data into a sparse matrix, using the same word-column mapping
test_matrix = vectorizer.transform(X_test)

In [13]:
model1 = LogisticRegression().fit(train_matrix, y_train)

C:\Users\yc1033514\Anaconda3\envs\py37\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [14]:
print("Coefficient: {}".format(model1.coef_))

Coefficient: [[ 0.0253789   0.17961501  0.12411337 ...  0.00105015  0.00102493
  -0.00050556]]


In [25]:
weights = model1.coef_
num_positive_weights = len([x for x in weights[0] if x > 0])
num_negative_weights = len([x for x in weights[0] if x < 0])

print("Number of positive weights: %s " % num_positive_weights)
print("Number of negative weights: %s " % num_negative_weights)

Number of positive weights: 37540 
Number of negative weights: 16287 


## Making predictions with logistic regression

Now that a model is trained, we can make predictions on the **test data**. In this section, we will explore this in the context of 3 examples in the test dataset.  We refer to this set of 3 examples as the **sample_test_data**.

In [26]:
sample_test_data = test_matrix[10:13]

We will now make a **class** prediction for the **sample_test_data**. The `sentiment_model` should predict **+1** if the sentiment is positive and **-1** if the sentiment is negative. Recall from the lecture that the **score** (sometimes called **margin**) for the logistic regression model  is defined as:

$$
\mbox{score}_i = \mathbf{w}^T h(\mathbf{x}_i)
$$ 

where $h(\mathbf{x}_i)$ represents the features for example $i$.  We will write some code to obtain the **scores** using Turi Create. For each row, the **score** (or margin) is a number in the range **[-inf, inf]**.

### Predicting sentiment

These scores can be used to make class predictions as follows:

$$
\hat{y} = 
\left\{
\begin{array}{ll}
      +1 & \mathbf{w}^T h(\mathbf{x}_i) > 0 \\
      -1 & \mathbf{w}^T h(\mathbf{x}_i) \leq 0 \\
\end{array} 
\right.
$$

Using scores, write code to calculate $\hat{y}$, the class predictions:

**Checkpoint**: Make sure your class predictions match with the one obtained from Turi Create.

### Probability predictions

Recall from the lectures that we can also calculate the probability predictions from the scores using:
$$
P(y_i = +1 | \mathbf{x}_i,\mathbf{w}) = \frac{1}{1 + \exp(-\mathbf{w}^T h(\mathbf{x}_i))}.
$$

Using the variable **scores** calculated previously, write code to calculate the probability that a sentiment is positive using the above formula. For each row, the probabilities should be a number in the range **[0, 1]**.

In [27]:
print("Predictions: {}".format(model1.predict(sample_test_data)))
print("Probability Prediction: {}".format(model1.predict_proba(sample_test_data)))

Predictions: [1 1 1]
Probability Prediction: [[3.96471438e-04 9.99603529e-01]
 [3.44356780e-05 9.99965564e-01]
 [5.72403784e-02 9.42759622e-01]]


## Compute accuracy of the classifier

We will now evaluate the accuracy of the trained classifier. Recall that the accuracy is given by


$$
\mbox{accuracy} = \frac{\mbox{# correctly classified examples}}{\mbox{# total examples}}
$$

This can be computed as follows:

* **Step 1:** Use the trained model to compute class predictions (**Hint:** Use the `predict` method)
* **Step 2:** Count the number of data points when the predicted class labels match the ground truth labels (called `true_labels` below).
* **Step 3:** Divide the total number of correct predictions by the total number of data points in the dataset.

Complete the function below to compute the classification accuracy:

In [29]:
from sklearn.metrics import accuracy_score

y_predict_train = model1.predict(train_matrix)
y_predict_test = model1.predict(test_matrix)

print("Accuracy of training set: {}".format(accuracy_score(y_train, y_predict_train)))
print("Accuracy of testing set: {}".format(accuracy_score(y_test, y_predict_test)))

Accuracy of training set: 0.9466720639275568
Accuracy of testing set: 0.9331654223261672


## Learn another classifier with fewer words

There were a lot of words in the model we trained above. We will now train a simpler logistic regression model using only a subset of words that occur in the reviews. For this assignment, we selected a 20 words to work with. These are:

In [30]:
significant_words = ['love', 'great', 'easy', 'old', 'little', 'perfect', 'loves', 
      'well', 'able', 'car', 'broke', 'less', 'even', 'waste', 'disappointed', 
      'work', 'product', 'money', 'would', 'return']

In [32]:
vectorizer_word_subset = CountVectorizer(vocabulary=significant_words) # limit to 20 words
train_matrix_word_subset = vectorizer_word_subset.fit_transform(X_train)
test_matrix_word_subset = vectorizer_word_subset.transform(X_test)

## Train a logistic regression model on a subset of data

In [34]:
model2 = LogisticRegression().fit(train_matrix_word_subset, y_train)

In [35]:
y_predict_train2 = model2.predict(train_matrix_word_subset)
y_predict_test2 = model2.predict(test_matrix_word_subset)

print("Accuracy of training set: {}".format(accuracy_score(y_train, y_predict_train2)))
print("Accuracy of testing set: {}".format(accuracy_score(y_test, y_predict_test2)))

Accuracy of training set: 0.8676321766703398
Accuracy of testing set: 0.8677400977481935


In [36]:
model2.coef_

array([[ 1.34909935,  0.94905156,  1.1646764 ,  0.09284139,  0.48257589,
         1.51325225,  1.73334205,  0.52827527,  0.21061647,  0.06148821,
        -1.69813896, -0.14612021, -0.53772699, -2.06513161, -2.32227148,
        -0.65417819, -0.31438264, -0.8778852 , -0.33132865, -2.07628285]])